In [5]:
import sys
from pathlib import Path

parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

In [6]:
import numpy as np
import cv2
import dlib
from collections import deque
import pickle
import time
from classes.image_processing import ImageProcessor
global_detector = dlib.get_frontal_face_detector()
global_predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')
global_sr_model = cv2.dnn_superres.DnnSuperResImpl_create()
ImageProcessor = ImageProcessor(global_detector, global_predictor)

In [7]:
#load the model
from keras.models import load_model
model = load_model('../eye_gaze_v28_24.h5')

In [8]:
# Screen dimensions
screen_width, screen_height = 1707, 960

# Grid size
grid_rows, grid_cols = 10, 10

# Initialize tracking for each cell
cell_durations = np.zeros((grid_rows, grid_cols))
cell_entry_times = np.full((grid_rows, grid_cols), None)

# Function to get cell based on gaze coordinates
def get_cell(gaze_x, gaze_y):
    cell_x = int(gaze_x // (screen_width / grid_cols))
    cell_y = int(gaze_y // (screen_height / grid_rows))
    return cell_x, cell_y

In [9]:
def update_gaze_duration(gaze_x, gaze_y, current_time):
    global cell_entry_times  # Access the global variable

    cell_x, cell_y = get_cell(gaze_x, gaze_y)
    
    # Check if it's a new cell
    if cell_entry_times[cell_y][cell_x] is None:
        # Update durations for previously entered cells
        for y in range(grid_rows):
            for x in range(grid_cols):
                if cell_entry_times[y][x] is not None:
                    duration = current_time - cell_entry_times[y][x]
                    cell_durations[y][x] += duration
                    cell_entry_times[y][x] = None  # Reset entry time
        
        # Update entry time for the new cell
        cell_entry_times[cell_y][cell_x] = current_time

In [10]:
def finalize_durations(final_time):
    global cell_durations, cell_entry_times  # Access the global variables
    
    for y in range(grid_rows):
        for x in range(grid_cols):
            if cell_entry_times[y][x] is not None:
                duration = final_time - cell_entry_times[y][x]
                cell_durations[y][x] += duration
                cell_entry_times[y][x] = None  # Reset for safety

In [11]:
# Initialize a queue to store gaze points (Change the number of points as needed)
n_points = 5
gaze_points_queue = deque(maxlen=n_points)
adjusted_gaze_points_queue = deque(maxlen=n_points)  # For adjusted points

# Screen dimensions
screenWidth = 1707
screenHeight = 960

with open('../pickel_files/adjustment_model.pkl', 'rb') as f:
    adjustment_model = pickle.load(f)

def moving_average(new_point, queue):
    queue.append(new_point)
    return [sum(x) / len(queue) for x in zip(*queue)]

cap = cv2.VideoCapture(0)
cv2.namedWindow('Gaze Tracking on Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking on Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

canvas = np.zeros((screenHeight, screenWidth, 3), dtype=np.uint8)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    combined_eyes = ImageProcessor.get_combined_eyes(frame, global_detector, global_predictor)

    if combined_eyes is not None:
        combined_eyes = np.expand_dims(combined_eyes, axis=0)
        predicted_gaze = model.predict(combined_eyes)[0][0]

        predicted_gaze = predicted_gaze[:2]

        gaze_x_scaled = int(predicted_gaze[0] * screenWidth)
        gaze_y_scaled = int(predicted_gaze[1] * screenHeight)

        current_time = time.time()
        update_gaze_duration(gaze_x_scaled, gaze_y_scaled, current_time)


        adjusted_pred = adjustment_model.predict(predicted_gaze.reshape(1, -1))[0]
        adjusted_x, adjusted_y = adjusted_pred[0] * screenWidth, adjusted_pred[1] * screenHeight

        # Apply moving average filter to both raw and adjusted gaze points
        gaze_x_smooth, gaze_y_smooth = moving_average((gaze_x_scaled, gaze_y_scaled), gaze_points_queue)
        adjusted_x_smooth, adjusted_y_smooth = moving_average((adjusted_x, adjusted_y), adjusted_gaze_points_queue)

        # Clamp to screen size after smoothing
        gaze_x_smooth = max(0, min(int(gaze_x_smooth), screenWidth - 1))
        gaze_y_smooth = max(0, min(int(gaze_y_smooth), screenHeight - 1))
        adjusted_x_smooth = max(0, min(int(adjusted_x_smooth), screenWidth - 1))
        adjusted_y_smooth = max(0, min(int(adjusted_y_smooth), screenHeight - 1))

        canvas.fill(0)  # Clear canvas for fresh drawing
        # Draw smoothed points
        cv2.circle(canvas, (gaze_x_smooth, gaze_y_smooth), 10, (0, 255, 0), -1)
        cv2.circle(canvas, (adjusted_x_smooth, adjusted_y_smooth), 10, (255, 0, 0), -1)

        cv2.imshow('Gaze Tracking on Canvas', canvas)

    cv2.imshow('Original Webcam Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        finalize_durations(time.time())
        break
                 
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 48ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 48ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 57ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 54ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 55ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 51ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 54ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 54ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 61ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 51ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 53ms/step
Blink Detected
Blink Detected
1/1 [==============================] - 0s 56ms/step
Blink Detected
Blink Detected
1/1 [===========

In [11]:
cell_durations

array([[0.        , 0.        , 0.        , 0.14511204, 0.        ,
        0.27668476, 0.        , 0.12998271, 1.34475374, 1.45825911],
       [0.52938485, 0.12802219, 0.13611221, 0.5161376 , 1.47502828,
        0.36724496, 0.26451731, 0.        , 0.53154659, 0.        ],
       [0.93596482, 0.91228056, 0.55291772, 0.51685619, 0.55631638,
        0.26182032, 0.        , 0.27046824, 0.        , 0.        ],
       [0.14457321, 1.74781919, 0.9439826 , 0.51101947, 1.43882298,
        0.1303544 , 0.        , 0.        , 0.39093637, 0.        ],
       [0.        , 0.90065527, 0.8250258 , 0.51492643, 0.15956354,
        0.28838038, 0.        , 0.1290431 , 0.1336534 , 0.13499141],
       [0.        , 0.65989184, 0.12576747, 0.41231441, 0.25762033,
        0.15214753, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.25242782, 0.1249609 , 0.13773322, 0.28025913,
        0.12524176, 0.        , 0.        , 0.32032609, 0.        ],
       [0.        , 0.        , 0.       

In [ ]:
import cv2
import numpy as np
from collections import deque
import pickle
from scipy.interpolate import CubicSpline

# Initialize a queue to store gaze points
n_points = 5
gaze_points_queue = deque(maxlen=n_points)

# Screen dimensions
screenWidth = 1707
screenHeight = 960

# Load the adjustment model
with open('../pickel_files/adjustment_model.pkl', 'rb') as f:
    adjustment_model = pickle.load(f)

# Initialize an empty canvas
canvas = np.zeros((screenHeight, screenWidth, 3), dtype=np.uint8)

# Global variable to store the previous bubble size for smoothing
previous_bubble_size = None

def draw_dynamic_confidence_bubble(canvas, points, base_size=7, smoothing_factor=0.8):
    global previous_bubble_size
    if not points:
        return canvas

    # Calculate mean of the points to find the center of the bubble
    mean_x, mean_y = np.mean(points, axis=0)
    
    # Calculate standard deviation as a measure of spread/confidence
    std_dev = np.std(points, axis=0)
    
    # Use the larger of the standard deviations to determine bubble size
    current_bubble_size = int(base_size + max(std_dev) * 5)  # Scale factor for visualization

    # Apply smoothing
    if previous_bubble_size is not None:
        bubble_size = int(previous_bubble_size * (1 - smoothing_factor) + current_bubble_size * smoothing_factor)
    else:
        bubble_size = current_bubble_size

    previous_bubble_size = bubble_size
    
    # Clear the canvas before drawing the new bubble
    canvas[:] = 0
    
    # Draw the bubble as a circle on the canvas
    cv2.circle(canvas, (int(mean_x), int(mean_y)), bubble_size, (0, 0, 255), 2)  # Red bubble with thickness of 2

    return canvas


cap = cv2.VideoCapture(0)
cv2.namedWindow('Gaze Tracking with Dynamic Confidence Bubble', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking with Dynamic Confidence Bubble', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    combined_eyes = get_combined_eyes(frame, global_detector, global_predictor)

    if combined_eyes is not None:
        combined_eyes = np.expand_dims(combined_eyes, axis=0)
        predicted_gaze = model.predict(combined_eyes)[0]

        predicted_gaze = predicted_gaze[:2]

        gaze_x_scaled = int(predicted_gaze[0] * screenWidth)
        gaze_y_scaled = int(predicted_gaze[1] * screenHeight)

        # Update gaze points queue
        gaze_points_queue.append((gaze_x_scaled, gaze_y_scaled))

        canvas_with_bubble = draw_dynamic_confidence_bubble(canvas, list(gaze_points_queue))
            
        # Display the canvas with the dynamic confidence bubble
        cv2.imshow('Gaze Tracking with Dynamic Confidence Bubble', canvas_with_bubble)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()